In [1]:
#setup

In [10]:
import pandas as pd

In [34]:
# Load data
matches_training_df = pd.read_parquet('matches_training.parquet')
offers_training_df = pd.read_parquet('offers_training.parquet')

# TRAIN, VALIDATION, TEST sets preprocessing

In [11]:
# Generic text preprocessing function to get lowercase and adjust all German nuances such as 'ü' and 'ß'

import unidecode

# Takes a string. Returns that string lowercased with accents removed.
def transform_txt(txt):
    txt = unidecode.unidecode(txt)
    txt = txt.lower()
    return txt

In [12]:
#preprocess brand so that it's just its first word
#this holds for ~97% of training matches , but still plenty of room for improvements

import re

def process_brand(txt):
    txt = re.sub('[^a-zA-Z ]+', '', txt)
    return transform_txt(txt).split()[0]

offers_training_df['brand'] = offers_training_df['brand'].apply(lambda x: transform_txt(x).split()[0])

NameError: name 'offers_training_df' is not defined

In [37]:
#preprocess price

offers_training_df['price'] = offers_training_df['price'].round(2)

In [38]:
# Split zalando and aboutyou offers into their own dataframes

zal_offers = offers_training_df[offers_training_df['shop']=='zalando']
aboutyou_offers = offers_training_df[offers_training_df['shop']=='aboutyou']

print(zal_offers.shape)
print(aboutyou_offers.shape)

zal_offers.dropna(inplace=True)
aboutyou_offers.dropna(inplace=True)

print(zal_offers.shape)
print(aboutyou_offers.shape)

(40904, 10)
(61980, 10)
(40876, 10)
(61978, 10)


/opt/anaconda3/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [39]:
# Preprocess description fields

In [13]:
# Takes a string of zalando description (messy key-values delimited by '$') and returns a clean list of words

def process_zal_descr(txt):
    bag = []
    
    for chunk in txt.split(' $ '):
        for word in chunk.split()[1:]:
            processed_word = transform_txt(word)
            if len(processed_word) >= 3 and processed_word.isalpha() and processed_word not in bag and processed_word not in ['bei', 'mit']:
                bag.append(processed_word)
    
    return bag

In [14]:
# Takes a string of aboutyou description (messy json and some html) and returns a clean list of words from the json part

import re

def process_aboutyou_json(txt):
    
    bag = []
    
    for json in re.findall(r"\[\"(.+?)\"\]", txt):
        for chunk in json.split():
            for word in re.findall(r"([\w]+)", chunk):
                processed_word = transform_txt(word)
                if len(processed_word) >= 3 and processed_word.isalpha() and processed_word not in bag and processed_word not in ['bei', 'mit']:
                    bag.append(processed_word)
    
    return bag

In [15]:
# Takes a string of aboutyou description (messy json and some html) and returns a clean list of words from the html part

def process_aboutyou_html(txt):
    bag = []
    
    for chunk in re.findall(r"<td>(.+?)</td>", txt):
        for word in re.findall(r"([\w]+)", chunk):
            processed_word = transform_txt(word)
            if len(processed_word) >= 3 and processed_word.isalpha() and processed_word not in bag and processed_word not in ['bei', 'mit']:
                bag.append(processed_word)
    
    return bag

In [16]:
# Takes an aboutyou description and returns the combined outputs of process_aboutyou_json() and process_aboutyou_html()

def process_aboutyou_descr(txt):
    return process_aboutyou_json(txt) + process_aboutyou_html(txt)

In [44]:
# Call functions to preprocess descriptions

zal_offers['description'] = zal_offers['description'].apply(process_zal_descr)
aboutyou_offers['description'] = aboutyou_offers['description'].apply(process_aboutyou_descr)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [45]:
# Preprocess color fields

In [17]:
# Takes a string of zalando color and returns a preprocessed string

import re

def process_zal_color(txt):

    #normalise
    txt = transform_txt(txt)

    #replace non-alphanumeric with space
    txt = re.sub('[^a-zA-Z]+', ' ', txt)

    #remove dunkel, hell
    txt = txt.replace("dunkel", "")
    txt = txt.replace("hell", "")

    return txt

In [18]:
# Takes a string of aboutyou color and returns a preprocessed version

def process_aboutyou_color(txt):
    
    #normalise
    txt = transform_txt(txt)

    #remove dunkel, hell
    txt = txt.replace("dunkel", "")
    txt = txt.replace("hell", "")

    return txt

In [48]:
# Call color preprocessing functions

zal_offers['color'] = zal_offers['color'].apply(process_zal_color)
aboutyou_offers['color'] = aboutyou_offers['color'].apply(process_aboutyou_color)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [19]:
# Takes a row with columns 'left_color' and 'right_color', returns True if any of left_color' in 'right_color'

def compare_color(x):

    for word in x['left_color'].split():
        if word in x['right_color']:
            return True
    return False

In [20]:
# Preprocess title fields

In [21]:
# Takes a string of zalando title and returns a preprocessed version

def process_zal_title(txt):
    #normalise
    return transform_txt(txt)

In [22]:
# Takes a string of aboutyou title and returns a preprocessed version
def process_aboutyou_title(txt):
    
    #normalise
    txt = transform_txt(txt)

    #remove apostrophes
    txt = txt.replace("'", "")

    return txt

In [53]:
# Call functions to preprocess title

zal_offers['title'] = zal_offers['title'].apply(process_zal_title)
aboutyou_offers['title'] = aboutyou_offers['title'].apply(process_aboutyou_title)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [23]:
# Takes a row with two 'left_title' and 'right_title' columns
# Returns True if any word is contained anywhere in the other

def title_words_in_common(x):
    for word in x['left_title'].split():
        if word in x['right_title']:
            return True
    for word in x['right_title'].split():
        if word in x['left_title']:
            return True
    return False

In [55]:
# Through an analysis of the expected size of brand cartesian products, combined with the blocking step of comparing
# description word counts, it was decided that three different train, validation, test sets would be separated by
# brands as below. The brands are listed
# below in ascending order of cartesian product

brands_1 = ['vaay',
'mons',
'zarkoperfume',
'hot',
'davida',
'burlington',
'bree',
'forever',
'olivia',
'polaroid',
'kessler',
'jette',
'veja',
'h.i.s',
'wheat',
'local',
'salamander',
'gore',
'verbenas',
'underprotection',
'luhta',
'flip*flop',
'estelle',
'garment',
'esme',
'fuchs',
'rukka',
'mother',
'kendall',
'ag',
'didriksons',
'farah',
'libertine-libertine',
'walkiddy',
'k-swiss',
'saucony',
'happy',
'dockers',
'fritzi',
'panama',
'envie',
'colmar',
'huf',
'club',
'jako',
'hust',
'herrlicher',
'burberry',
'call',
'denim',
'camano',
'peak',
'jacky',
'moves',
'primigi',
'bruuns',
'gestuz',
'mennace',
'bjorn',
'rosemunde',
'quiksilver',
'swarovski',
'free',
'bullboxer',
'zizzi',
'ted',
'etam',
'rich']

brands_2=[
'liu',
'more',
'kaffe',
'ellesse',
'mamalicious',
'lascana']

brands_3 = [
'selected',
'guess',
'pieces']

too_big=[
'vero']

In [24]:
# Takes a row with two 'left_description' and 'right_description' columns (lists).
# Returns a count of the number of words they have in common

def desc_words_in_common(x):
    counter = 0
    for word in x['left_description']:
        if word in x['right_description']:
            counter += 1
    return counter

In [25]:
# Takes a row with two 'left_price' and 'right_price' columns floats.
# Returns the absolute difference in price

def price_diff(x):
    return abs(x['left_price'] - x['right_price'])

In [58]:
# A loop to create three train, validation and test sets based on brand groupings defined above

from sklearn.model_selection import train_test_split
import os

# only columns of interest for now
cols = ['offer_id', 'brand', 'title', 'color', 'description', 'price']

# Loop from 1 to 3 since there will three train, validation and test sets
for i in range(1, 4):
    
    # Create train dataframe
    globals()[f'train_df_{i}'] = pd.DataFrame()
    
    brand_group = f'brands_{i}'
    for brand in globals()[brand_group]:
        # Only take columns of interest
        temp_zal = zal_offers[zal_offers['brand'] == brand][cols]
        temp_aboutyou = aboutyou_offers[aboutyou_offers['brand'] == brand][cols]
        
        # Format column names as per deepmatcher requirements
        temp_zal = temp_zal.add_prefix('left_')
        temp_aboutyou = temp_aboutyou.add_prefix('right_')
        
        # Create an intra-brand cartesian product
        brand_cross = temp_zal.merge(temp_aboutyou, how='cross')
        
        # Prune from cartesian product pairs with more than 2.5 euro in price difference
        if not brand_cross.empty:
            brand_cross['price_diff'] = brand_cross.apply(price_diff, axis=1)
            brand_cross = brand_cross[brand_cross['price_diff'] <= 2.5]
        
        if not brand_cross.empty:
            brand_cross['color_compare'] = brand_cross.apply(compare_color, axis=1)
            brand_cross = brand_cross[brand_cross['color_compare'] == True]
            
        if not brand_cross.empty:
            brand_cross['title_match'] = brand_cross.apply(title_words_in_common, axis=1)
            brand_cross = brand_cross[brand_cross['title_match'] == True]
        
        # Prune from cartesian product pairs with fewer than six words in common in the description
        if not brand_cross.empty:
            brand_cross['words_in_common'] = brand_cross.apply(desc_words_in_common, axis=1)
            brand_cross = brand_cross[brand_cross['words_in_common'] > 5]
        
        # Append the brand's pruned cartesian product to train_df_{i}
        globals()[f'train_df_{i}'] = globals()[f'train_df_{i}'].append(brand_cross, ignore_index=True)
        
    #  Make the 'label' column in train_df_{i}, indicating ground-truth matches
    globals()[f'train_df_{i}'] = globals()[f'train_df_{i}'].merge(matches_training_df
                              ,how='left'
                              ,left_on=['left_offer_id', 'right_offer_id']
                              ,right_on=['zalando', 'aboutyou'])
    globals()[f'train_df_{i}'].insert(0, 'label', globals()[f'train_df_{i}']['zalando'].notnull().astype(int))
    
    # Drop unnecessary columns in train_df_{i}
    drop_cols = ['left_offer_id', 'right_offer_id', 'zalando', 'aboutyou', 'brand', 'words_in_common',
                'left_description', 'right_description', 'price_diff', 'color_compare', 'title_match']
    globals()[f'train_df_{i}'] = globals()[f'train_df_{i}'].drop(drop_cols, 1)
    
    # Update index name in train_df_{i}
    globals()[f'train_df_{i}'].index.names = ['id']
    
    # Split train_df_{i} into train_df, valid_df and test_df
    train_df, holder_df = train_test_split(globals()[f'train_df_{i}'], train_size=0.66, random_state=42, stratify=globals()[f'train_df_{i}']['label'])
    valid_df, test_df = train_test_split(holder_df, train_size=0.66, random_state=42, stratify=holder_df['label'])
    
    # Save train_df, valid_df and test_df as CSVs
    cwd = str(os.getcwd())
    train_df.to_csv(cwd + f'/train_{i}.csv')
    valid_df.to_csv(cwd + f'/validation_{i}.csv')
    test_df.to_csv(cwd + f'/test_{i}.csv')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [59]:
train_df_1.shape

(5042, 9)

In [60]:
train_df_2.shape

(10300, 9)

In [61]:
train_df_3.columns

Index(['label', 'left_brand', 'left_title', 'left_color', 'left_price',
       'right_brand', 'right_title', 'right_color', 'right_price'],
      dtype='object')

In [62]:
train_df_3.shape

(24345, 9)

# UNLABELLED set preprocessing

In [1]:
#Carry out same steps as above for unlabelled data

In [26]:
offers_test_df = pd.read_parquet('offers_test.parquet')

In [27]:
offers_test_df['brand'] = offers_test_df['brand'].apply(process_brand)

In [28]:
offers_test_df['price'] = offers_test_df['price'].round(2)

In [29]:
# Split zalando and aboutyou offers into their own dataframes

zal_offers = offers_test_df[offers_test_df['shop']=='zalando']
aboutyou_offers = offers_test_df[offers_test_df['shop']=='aboutyou']

print(zal_offers.shape)
print(aboutyou_offers.shape)

zal_offers.dropna(inplace=True)
aboutyou_offers.dropna(inplace=True)

print(zal_offers.shape)
print(aboutyou_offers.shape)

(36636, 10)
(70105, 10)
(36578, 10)
(70090, 10)


/opt/anaconda3/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [30]:
zal_offers['description'] = zal_offers['description'].apply(process_zal_descr)
aboutyou_offers['description'] = aboutyou_offers['description'].apply(process_aboutyou_descr)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
zal_offers['color'] = zal_offers['color'].apply(process_zal_color)
aboutyou_offers['color'] = aboutyou_offers['color'].apply(process_aboutyou_color)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
zal_offers['title'] = zal_offers['title'].apply(process_zal_title)
aboutyou_offers['title'] = aboutyou_offers['title'].apply(process_aboutyou_title)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
# The equivalent of the above train loop, for the test set

import os

test_df = pd.DataFrame()

# Get a list of brands to look at
zal_brands = zal_offers['brand'].unique()
aboutyou_brands = aboutyou_offers['brand'].unique()
brand_list = list(set(zal_brands).intersection(aboutyou_brands))
# Removing tom becuase it is causing the notebook to crash
brand_list.remove('tom')

# only columns of interest for now
cols = ['offer_id', 'brand', 'title', 'color', 'description', 'price']

for brand in brand_list:
    
    # Only take columns of interest
    temp_zal = zal_offers[zal_offers['brand'] == brand][cols]
    temp_aboutyou = aboutyou_offers[aboutyou_offers['brand'] == brand][cols]
        
    # Format column names as per deepmatcher requirements
    temp_zal = temp_zal.add_prefix('left_')
    temp_aboutyou = temp_aboutyou.add_prefix('right_')
        
    # Create an intra-brand cartesian product
    brand_cross = temp_zal.merge(temp_aboutyou, how='cross')
    
    drop_cols = ['left_description', 'right_description']
        
    # Prune from cartesian product pairs with more than 6 euro in price difference
    if not brand_cross.empty:
        brand_cross['price_diff'] = brand_cross.apply(price_diff, axis=1)
        brand_cross = brand_cross[brand_cross['price_diff'] <= 2.5]
        drop_cols.append('price_diff')
        
    if not brand_cross.empty:
        brand_cross['color_compare'] = brand_cross.apply(compare_color, axis=1)
        brand_cross = brand_cross[brand_cross['color_compare'] == True]
        drop_cols.append('color_compare')
            
    if not brand_cross.empty:
        brand_cross['title_match'] = brand_cross.apply(title_words_in_common, axis=1)
        brand_cross = brand_cross[brand_cross['title_match'] == True]
        drop_cols.append('title_match')
        
    # Prune from cartesian product pairs with fewer than six words in common in the description
    if not brand_cross.empty:
        brand_cross['words_in_common'] = brand_cross.apply(desc_words_in_common, axis=1)
        brand_cross = brand_cross[brand_cross['words_in_common'] > 5]
        drop_cols.append('words_in_common')
    
    # Drop unnecessary columns
    brand_cross = brand_cross.drop(drop_cols, 1)
    
    test_df = test_df.append(brand_cross, ignore_index=True)
    
test_offer_ids = test_df[['left_offer_id', 'right_offer_id']]
test_df = test_df.drop(['left_offer_id', 'right_offer_id'], 1)
    
# Update index name
test_df.index.names = ['id']
test_offer_ids.index.names = ['id']
    
# Save as CSVs
cwd = str(os.getcwd())
test_df.to_csv(cwd + '/test_set.csv')
test_offer_ids.to_csv(cwd + '/test_offer_ids.csv')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [34]:
test_df.shape

(37541, 8)

In [35]:
test_offer_ids.shape

(37541, 2)